In [137]:
import pandas as pd
import pickle, math
import numpy as np
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import MinMaxScaler

In [146]:

# Unpickle the DataFrame
with open('sensor_data.pkl', 'rb') as file:
    sensor_df = pickle.load(file)

sensor_df["pv_change"] = sensor_df["PV0"] - sensor_df["PV1"]
sensor_df["T_change"] = sensor_df["Temp_Top"] - sensor_df["Temp_Bot"]
sensor_df

,Time,PV0,PV1,Temp_Top,Temp_Bot,pv_change,T_change
0,1.691166e+09,0.416979,0.005334,21.1250,21.6250,0.411646,-0.5000
1,1.691167e+09,0.416997,0.005308,20.6875,21.0000,0.411689,-0.3125
2,1.691168e+09,0.416985,0.005280,21.8125,21.9375,0.411705,-0.1250
3,1.691168e+09,0.416990,0.005306,22.2500,22.3125,0.411684,-0.0625
4,1.691169e+09,0.417011,0.005335,22.5000,22.5625,0.411676,-0.0625
...,...,...,...,...,...,...,...
7860,1.695904e+09,0.587452,0.486187,20.1250,27.4375,0.101265,-7.3125
7861,1.695905e+09,0.587445,0.486258,20.3750,27.2500,0.101187,-6.8750
7862,1.695905e+09,0.587498,0.486293,20.6250,26.9375,0.101205,-6.3125
7863,1.695906e+09,0.587526,0.486342,20.8125,26.6875,0.101183,-5.8750


In [158]:
predict = "pv_change"

X = sensor_df.drop(["PV1", predict], axis=1)
y = sensor_df[predict]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

# Create a Min-Max scaler
scaler = MinMaxScaler()

# Fit the scaler to your training data and transform both training and test data
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)


In [159]:
clf = LinearRegression()
clf.fit(X_train_normalized, y_train)
accuracy = clf.score(X_test_normalized, y_test)


# Get the coefficients
coefficients = clf.coef_

# Print out the coefficients
print(f"Accuracy: {accuracy}")
print("Coefficients:")
for i, coef in enumerate(coefficients):
    print(f"Feature {i}: {coef}")

Accuracy: 0.0068718414504732594
Coefficients:
Feature 0: 0.017275371436162146
Feature 1: 0.04465535871564252
Feature 2: 0.00837794218592959
Feature 3: 0.012035443652705445
Feature 4: -0.010136451353282762


In [118]:
forecast_col = "pv_change"
sensor_df.fillna(-99999, inplace=True)

forecast_out = int(math.ceil(0.10*len(sensor_df)))

sensor_df["label"] = sensor_df[forecast_col].shift(-forecast_out)
sensor_df.dropna(inplace=True)

In [126]:
# Assuming X is your NumPy array and you want to normalize the first column
X = np.array(sensor_df.drop(["label", "Time"], axis=1))
y = np.array(sensor_df["label"])
print(X)
# # Calculate the min and max of the first column
# min_value = np.min(X[:, 0])
# max_value = np.max(X[:, 0])

# # Calculate the min-max normalization for the first column
# normalized_first_column = (X[:, 0] - min_value) / (max_value - min_value)

# # Replace the first column of X with the normalized values
# X[:, 0] = normalized_first_column

X = preprocessing.scale(X)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

[[ 3.59832800e-03  3.58184800e-03  2.11250000e+01  2.16250000e+01
   1.64800000e-05 -5.00000000e-01]
 [ 3.60916100e-03  3.60702500e-03  2.06875000e+01  2.10000000e+01
   2.13600000e-06 -3.12500000e-01]
 [ 3.59634400e-03  3.58886700e-03  2.18125000e+01  2.19375000e+01
   7.47700000e-06 -1.25000000e-01]
 ...
 [ 5.71004629e-01  3.85078728e-01  2.00000000e+01  1.86875000e+01
   1.85925901e-01  1.31250000e+00]
 [ 5.71014583e-01  3.84985059e-01  2.00000000e+01  1.86250000e+01
   1.86029524e-01  1.37500000e+00]
 [ 5.71054399e-01  3.85129690e-01  1.99375000e+01  1.85625000e+01
   1.85924709e-01  1.37500000e+00]]


In [124]:
clf = LinearRegression()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
accuracy

0.26216887187270144

In [127]:
24*60*60

86400